In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.plotting import figure, output_file, show, save
from bokeh.io import output_notebook
pd.options.display.float_format = '{:,.2f}'.format
output_notebook()

Loading BokehJS ...

In [2]:
# 2017 Reform
path_name = 'C:\\Users\\derrick.choe\\Tax-Calculator\\taxcalc\\reforms\\2017_law.json'
jfile = open(path_name, 'r')
jtext = jfile.read()
jfile.close()
reform_old = tc.Calculator._read_json_policy_reform_text(jtext, {}, {})

pol_old = tc.Policy()
pol_old.implement_reform(reform_old)

#Baseline Calculator
calc_old=tc.Calculator(records = tc.Records(), policy = pol_old, verbose = False)
calc_old.advance_to_year(2018)
calc_old.calc_all()

In [3]:
# 2017 Reform, no charitable deduction
reform_old_nochar = copy.deepcopy(reform_old)
reform_old_nochar[2018]['_ID_Charity_hc'] = [1]

pol_old_nochar = tc.Policy()
pol_old_nochar.implement_reform(reform_old_nochar)

calc_old_nochar = tc.Calculator(records = tc.Records(), policy = pol_old_nochar, verbose = False)
calc_old_nochar.advance_to_year(2018)
calc_old_nochar.calc_all()

In [4]:
calc_new = tc.Calculator(records = tc.Records(), policy = tc.Policy(), verbose = False)
calc_new.advance_to_year(2018)
calc_new.calc_all()

In [5]:
# TCJA, no charitable deduction
reform_new_nochar = {2018 : {}}
reform_new_nochar[2018]['_ID_Charity_hc'] = [1]

pol_new_nochar = tc.Policy()
pol_new_nochar.implement_reform(reform_new_nochar)

calc_new_nochar = tc.Calculator(records = tc.Records(), policy = pol_new_nochar, verbose = False)
calc_new_nochar.advance_to_year(2018)
calc_new_nochar.calc_all()

In [6]:
vlist = ['c00100', 'iitax', 'standard', 'c04470', 'c19200', 'c19700', 'n24', 'c07220', 'c11070', 's006']

In [7]:
odf_full = calc_old.dataframe(vlist)
ndf_full = calc_new.dataframe(vlist)

In [8]:
def getstat(df, title, var = 's006'):
    print(title + ": " + "{:,.0f}".format((df[var].sum())))

In [9]:
def w_avg(df, var):
    return (df[var] * df['s006']).sum() / df['s006'].sum()

# Section 1: Number of Filers

## 1a. Number of Tax Filers with Tax Liability > \$0

In [10]:
getstat(odf_full[odf_full['iitax'] > 0], 'Pre-TCJA')
getstat(ndf_full[ndf_full['iitax'] > 0], 'TCJA')

Pre-TCJA: 103,354,373
TCJA: 97,069,932


## 1b. Number of Tax Filers Claiming Standard, Itemized Deduction

In [11]:
for df in [(odf_full, 'Pre-TCJA'), (ndf_full, 'TCJA')]:
    getstat(df[0][(df[0]['iitax'] > 0) & (df[0]['standard'] > 0)], df[1] + ', Standard Deduction, Positive Income Tax Liability')
    getstat(df[0][(df[0]['iitax'] > 0) & (df[0]['c04470'] > 0)], df[1] + ', Itemized Deduction, Positive Income Tax Liability')
    getstat(df[0][df[0]['standard'] > 0], df[1] + ', Standard Deduction, All Tax Filers')
    getstat(df[0][df[0]['c04470'] > 0], df[1] + ', Itemized Deduction, All Tax Filers')

Pre-TCJA, Standard Deduction, Positive Income Tax Liability: 60,869,564
Pre-TCJA, Itemized Deduction, Positive Income Tax Liability: 42,466,657
Pre-TCJA, Standard Deduction, All Tax Filers: 127,618,535
Pre-TCJA, Itemized Deduction, All Tax Filers: 45,164,447
TCJA, Standard Deduction, Positive Income Tax Liability: 78,528,593
TCJA, Itemized Deduction, Positive Income Tax Liability: 18,517,343
TCJA, Standard Deduction, All Tax Filers: 153,399,973
TCJA, Itemized Deduction, All Tax Filers: 19,396,400


In [12]:
odf_posagi = copy.deepcopy(odf_full[odf_full['c00100'] > 0])
ndf_posagi = copy.deepcopy(ndf_full[odf_full['c00100'] > 0])

## 1c. Number Claiming Standard, Itemized Deductions by AGI Decile

In [13]:
numbin = 10
tc.add_quantile_bins(odf_posagi, income_measure = 'c00100', num_bins = numbin)
odf_posagi = odf_posagi.sort_index()
ndf_posagi['bins'] = odf_posagi['bins']
decile_dict = {}
for df in [(odf_posagi, 'Pre-TCJA'), (ndf_posagi, 'TCJA')]:
    for i in range(1, numbin + 1):
        bindf = df[0][df[0]['bins'] == i]
        decile_dict.setdefault(df[1] + ', Standard Deduction', [])
        decile_dict[df[1] + ', Standard Deduction'].append(bindf[bindf['standard'] > 0]['s006'].sum())
        decile_dict.setdefault(df[1] + ', Itemized Deduction', [])
        decile_dict[df[1] + ', Itemized Deduction'].append(bindf[bindf['c04470'] > 0]['s006'].sum())
decile_df = pd.DataFrame(decile_dict)
decile_df = decile_df.append(decile_df.sum(numeric_only=True), ignore_index=True)

decile_df.index = list(zip(range(1,11), 
                           odf_posagi.groupby('bins')['c00100'].min(), 
                           odf_posagi.groupby('bins')['c00100'].max())) + ['Total']

decile_df

,"Pre-TCJA, Itemized Deduction","Pre-TCJA, Standard Deduction","TCJA, Itemized Deduction","TCJA, Standard Deduction"
"(1, 0.496695789163, 4377.20002579)","3,814.55","16,095,339.48","19,652.40","16,095,328.84"
"(2, 4377.62818215, 10925.2370646)","21,155.82","16,093,592.51","13,961.97","16,101,340.02"
"(3, 10925.2370646, 17561.6869681)","532,534.22","15,587,796.42","156,362.51","15,964,277.37"
"(4, 17561.6869681, 26551.5962828)","1,075,383.14","15,044,814.98","369,218.00","15,749,017.25"
"(5, 26551.5986875, 37445.9949212)","2,007,518.29","14,111,940.89","611,336.68","15,505,921.83"
"(6, 37445.9949212, 51522.1260651)","3,373,580.96","12,745,360.58","1,008,296.28","15,109,029.61"
"(7, 51523.6124798, 69819.6364485)","5,356,757.03","10,756,904.12","1,866,314.53","14,248,946.23"
"(8, 69820.5672485, 98062.4259455)","7,699,349.45","8,419,487.16","2,927,089.89","13,192,530.53"
"(9, 98065.2950764, 148003.050189)","10,814,196.70","5,306,380.43","4,162,447.16","11,958,019.96"
"(10, 148003.050189, 197584873.031)","14,280,078.72","1,839,691.77","8,261,521.85","7,858,454.80"


## 1d. Number Claiming Interest Deduction

In [14]:
decile_dict = {}
for df in [(odf_posagi, 'Pre-TCJA'), (ndf_posagi, 'TCJA')]:
    for i in range(1, numbin + 1):
        bindf = df[0][df[0]['bins'] == i]
        decile_dict.setdefault(df[1] + ', Interest Deduction', [])
        decile_dict[df[1] + ', Interest Deduction'].append(bindf[(bindf['c19200'] > 0) & (bindf['c04470'] > 0)]['s006'].sum())
decile_df = pd.DataFrame(decile_dict)
decile_df = decile_df.append(decile_df.sum(numeric_only=True), ignore_index=True)
decile_df.index = list(zip(range(1,11), 
                           odf_posagi.groupby('bins')['c00100'].min(), 
                           odf_posagi.groupby('bins')['c00100'].max())) + ['Total']
decile_df

,"Pre-TCJA, Interest Deduction","TCJA, Interest Deduction"
"(1, 0.496695789163, 4377.20002579)",0.00,"4,345.93"
"(2, 4377.62818215, 10925.2370646)","13,339.02","10,140.00"
"(3, 10925.2370646, 17561.6869681)","290,814.17","91,171.79"
"(4, 17561.6869681, 26551.5962828)","611,190.42","204,622.95"
"(5, 26551.5986875, 37445.9949212)","1,256,671.79","398,574.26"
"(6, 37445.9949212, 51522.1260651)","2,293,844.22","729,460.96"
"(7, 51523.6124798, 69819.6364485)","4,001,049.69","1,530,141.46"
"(8, 69820.5672485, 98062.4259455)","6,115,719.36","2,474,398.55"
"(9, 98065.2950764, 148003.050189)","9,133,761.45","3,680,240.77"
"(10, 148003.050189, 197584873.031)","12,170,547.80","7,551,887.88"


## 1e. Number Claiming Charitable Deduction

In [15]:
decile_dict = {}
for df in [(odf_posagi, 'Pre-TCJA'), (ndf_posagi, 'TCJA')]:
    for i in range(1, numbin + 1):
        bindf = df[0][df[0]['bins'] == i]
        decile_dict.setdefault(df[1] + ', Charity Deduction', [])
        decile_dict[df[1] + ', Charity Deduction'].append(bindf[(bindf['c19700'] > 0) & (bindf['c04470'] > 0)]['s006'].sum())
decile_df = pd.DataFrame(decile_dict)
decile_df = decile_df.append(decile_df.sum(numeric_only=True), ignore_index=True)
decile_df.index = list(zip(range(1,11), 
                           odf_posagi.groupby('bins')['c00100'].min(), 
                           odf_posagi.groupby('bins')['c00100'].max())) + ['Total']
decile_df

,"Pre-TCJA, Charity Deduction","TCJA, Charity Deduction"
"(1, 0.496695789163, 4377.20002579)",0.00,"7,037.37"
"(2, 4377.62818215, 10925.2370646)","10,754.33","6,770.70"
"(3, 10925.2370646, 17561.6869681)","321,271.86","85,553.92"
"(4, 17561.6869681, 26551.5962828)","685,968.27","212,481.78"
"(5, 26551.5986875, 37445.9949212)","1,430,103.42","454,200.76"
"(6, 37445.9949212, 51522.1260651)","2,457,727.73","787,780.94"
"(7, 51523.6124798, 69819.6364485)","4,082,518.79","1,456,580.70"
"(8, 69820.5672485, 98062.4259455)","6,309,127.95","2,489,909.30"
"(9, 98065.2950764, 148003.050189)","9,259,098.23","3,677,894.65"
"(10, 148003.050189, 197584873.031)","13,086,027.57","7,756,719.73"


# Section 2: Marginal Tax Rates on Labor

## 2a. Effective Marginal Tax Rate on Labor

In [16]:
odf_full['mtr'] = calc_old.mtr()[1]
ndf_full['mtr'] = calc_new.mtr()[1]

In [17]:
print('Pre-TCJA, Average EMTR: ' + str(w_avg(odf_full, 'mtr')))
print('TCJA, Average EMTR: ' + str(w_avg(ndf_full, 'mtr')))

Pre-TCJA, Average EMTR: 0.11580142502096552
TCJA, Average EMTR: 0.09189758044937907


# Section 3: The Child Tax Credit

## 3a./3b. Number of Taxpayers Claiming CTC (Pre-TCJA)

In [18]:
getstat(odf_full[(odf_full['iitax'] > 0) & ((odf_full['c07220'] > 0) | (odf_full['c11070'] > 0))], 'Pre-TCJA: Positive Tax Liability')
getstat(odf_full[(odf_full['iitax'] <= 0) & ((odf_full['c07220'] > 0) | (odf_full['c11070'] > 0))], 'Pre-TCJA: 0/Negative Tax Liability')
getstat(odf_full[(odf_full['c07220'] > 0) | (odf_full['c11070'] > 0)], 'Pre-TCJA: All Tax Filers')

Pre-TCJA: Positive Tax Liability: 14,103,647
Pre-TCJA: 0/Negative Tax Liability: 20,169,227
Pre-TCJA: All Tax Filers: 34,272,874


## 3c./3d. Number of Taxpayers Claiming CTC (TCJA)

In [19]:
getstat(ndf_full[(ndf_full['iitax'] > 0) & ((ndf_full['c07220'] > 0) | (ndf_full['c11070'] > 0))], 'TCJA: Positive Tax Liability')
getstat(ndf_full[(ndf_full['iitax'] <= 0) & ((ndf_full['c07220'] > 0) | (ndf_full['c11070'] > 0))], 'TCJA: 0/Negative Tax Liability')
getstat(ndf_full[(ndf_full['c07220'] > 0) | (ndf_full['c11070'] > 0)], 'TCJA: All Tax Filers')

TCJA: Positive Tax Liability: 18,016,594
TCJA: 0/Negative Tax Liability: 22,485,246
TCJA: All Tax Filers: 40,501,840


## 3e. Average Tax Burden by Family Size, AGI < \$250,000

In [20]:
ch_dict = {}
for df in [(odf_full[(odf_full['c00100'] < 250000)], 'Pre-TCJA'), (ndf_full[(ndf_full['c00100'] < 250000)], 'TCJA')]:
    for i in range(0, 4):
        bindf = df[0][df[0]['n24'] == i]
        taxsum = (bindf['iitax'] * bindf['s006']).sum()
        incsum = (bindf['c00100'] * bindf['s006']).sum()
        ch_dict.setdefault(df[1] + ', Average Income Tax', [])
        ch_dict[df[1] + ', Average Income Tax'].append(taxsum / bindf['s006'].sum())
        ch_dict.setdefault(df[1] + ', Average Tax Rate', [])
        ch_dict[df[1] + ', Average Tax Rate'].append(taxsum / incsum)
decile_df = pd.DataFrame(ch_dict)
decile_df

,"Pre-TCJA, Average Income Tax","Pre-TCJA, Average Tax Rate","TCJA, Average Income Tax","TCJA, Average Tax Rate"
0,"4,621.28",0.11,"4,071.30",0.09
1,"3,406.54",0.06,"2,258.38",0.04
2,"2,858.19",0.04,"1,297.19",0.02
3,"1,411.65",0.02,-397.14,-0.01


# Section 4: Charitable Giving

## 4a. Number of Taxpayers Claiming Charitable Deduction

In [21]:
getstat(odf_full[(odf_full['c19700'] > 0) & (odf_full['c04470'] > 0)], 'Pre-TCJA, All Filers, Number Taking Charitable Deduction')
getstat(odf_full[(odf_full['iitax'] > 0) & (odf_full['c19700'] > 0) & (odf_full['c04470'] > 0)], 'Pre-TCJA, Positive Tax Liability, Number Taking Charitable Deduction')
getstat(ndf_full[(ndf_full['c19700'] > 0) & (ndf_full['c04470'] > 0)], 'TCJA, All Filers, Number Taking Charitable Deduction')
getstat(ndf_full[(ndf_full['iitax'] > 0) & (ndf_full['c19700'] > 0) & (ndf_full['c04470'] > 0)], 'TCJA, Positive Tax Liability, Number Taking Charitable Deduction')


Pre-TCJA, All Filers, Number Taking Charitable Deduction: 37,642,598
Pre-TCJA, Positive Tax Liability, Number Taking Charitable Deduction: 35,796,801
TCJA, All Filers, Number Taking Charitable Deduction: 16,935,096
TCJA, Positive Tax Liability, Number Taking Charitable Deduction: 16,358,026


## 4b. Average Tax Breaks Through Charitable Deduction

In [22]:
odf_full['ch_iidiff'] = odf_full['iitax'] - calc_old_nochar.array('iitax')
ndf_full['ch_iidiff'] = ndf_full['iitax'] - calc_new_nochar.array('iitax')
print('Pre-TCJA: ' + str(-w_avg(odf_full, 'ch_iidiff')))
print('TCJA: ' + str(-w_avg(ndf_full, 'ch_iidiff')))

Pre-TCJA: 318.6706840673743
TCJA: 207.87353512207795


## 4c. Share of Taxpayers Claiming Itemized Deduction

In [23]:
pd.options.display.float_format = '{:,.6f}'.format

decile_dict = {}
for df in [(odf_posagi, 'Pre-TCJA'), (ndf_posagi, 'TCJA')]:
    for i in range(1, numbin + 1):
        bindf = df[0][df[0]['bins'] == i]
        decile_dict.setdefault(df[1] + ', Share Claiming Itemized Deductions', [])
        decile_dict[df[1] + ', Share Claiming Itemized Deductions'].append(bindf[bindf['c04470'] > 0]['s006'].sum() / bindf['s006'].sum())
decile_df = pd.DataFrame(decile_dict)
decile_df.index = list(zip(range(1,11), 
                           odf_posagi.groupby('bins')['c00100'].min(), 
                           odf_posagi.groupby('bins')['c00100'].max()))
decile_df

,"Pre-TCJA, Share Claiming Itemized Deductions","TCJA, Share Claiming Itemized Deductions"
"(1, 0.496695789163, 4377.20002579)",0.000237,0.001219
"(2, 4377.62818215, 10925.2370646)",0.001312,0.000866
"(3, 10925.2370646, 17561.6869681)",0.033034,0.009699
"(4, 17561.6869681, 26551.5962828)",0.066710,0.022904
"(5, 26551.5986875, 37445.9949212)",0.124527,0.037921
"(6, 37445.9949212, 51522.1260651)",0.209292,0.062553
"(7, 51523.6124798, 69819.6364485)",0.332304,0.115776
"(8, 69820.5672485, 98062.4259455)",0.477590,0.181567
"(9, 98065.2950764, 148003.050189)",0.670826,0.258205
"(10, 148003.050189, 197584873.031)",0.885798,0.512465


## 4d. Distribution of Share of Previous Charitable Deduction Takers Switching to Standard

In [24]:
pd.options.display.float_format = '{:,.2f}'.format

sdf = copy.deepcopy(odf_posagi)
nbins = 100
sdf['switch'] = np.where((sdf['c19700'] > 0) & (sdf['c04470'] > 0) & (ndf_posagi['standard'] > 0), sdf['s006'], 0)
sdf['char'] = np.where((sdf['c19700'] > 0) & (sdf['c04470'] > 0), sdf['s006'], 0.001)
tc.add_quantile_bins(sdf, income_measure='c00100', num_bins= nbins)
quantile_dict = {}
for i in range(1, nbins + 1):
    bindf = sdf[sdf['bins'] == i]
    quantile_dict.setdefault('Share of Charity Deduction Takers Switching to Standard', [])
    quantile_dict['Share of Charity Deduction Takers Switching to Standard'].append(bindf['switch'].sum()/ bindf['char'].sum())
quantile_df = pd.DataFrame(quantile_dict)
quantile_df.index = list(zip(range(1,nbins + 1), 
                           sdf.groupby('bins')['c00100'].min(), 
                           sdf.groupby('bins')['c00100'].max()))

f = figure()
f.vbar(x = list(range(1,nbins + 1)),  width = 0.5, top = quantile_df['Share of Charity Deduction Takers Switching to Standard'])
show(f)

## 4e. Estimated Impact on Actual Giving (Varying Elasticities, Copied From Charity Project)

In [25]:
print('Income Elasticity of 0.5, Price Elasticity of -1, No ETI: ' + '{:,.2f}'.format(-23459781496.25))
print('Income Elasticity of 0.5, Price Elasticity of -1, ETI of 0.25: ' + '{:,.2f}'.format(-20463936248.964443))
print('Income Elast of 0.5, Price Elast of -1 for After-tax Incomes Over $200k, -0.5 for Under, ETI of 0.25: ' + '{:,.2f}'.format(-11829356913.201384))


Income Elasticity of 0.5, Price Elasticity of -1, No ETI: -23,459,781,496.25
Income Elasticity of 0.5, Price Elasticity of -1, ETI of 0.25: -20,463,936,248.96
Income Elast of 0.5, Price Elast of -1 for After-tax Incomes Over $200k, -0.5 for Under, ETI of 0.25: -11,829,356,913.20


## 4f. Required GDP Effect (To Break Even) **still in progress, but so far it looks high (5% +)

# Section 5: Housing

# 5a. Number of Taxpayers Previously Claiming Interest Deduction Switching to the Standard

In [26]:
sdf = copy.deepcopy(odf_posagi)
sdf['newiitax'] = ndf_posagi['iitax']
nbins = 10
sdf['switch'] = np.where((sdf['c19200'] > 0) & (sdf['c04470'] > 0) & (ndf_posagi['standard'] > 0), sdf['s006'], 0)
sdf['intr'] = np.where((sdf['c19200'] > 0) & (sdf['c04470'] > 0), sdf['s006'], 1e-99)
tc.add_quantile_bins(sdf, income_measure='c00100', num_bins= nbins)
quantile_dict = {}
for i in range(1, nbins + 1):
    bindf = sdf[sdf['bins'] == i]
    quantile_dict.setdefault('Share of Interest Deduction Takers Switching to Standard', [])
    quantile_dict['Share of Interest Deduction Takers Switching to Standard'].append(bindf['switch'].sum()/ bindf['intr'].sum())
    quantile_dict.setdefault('Number of Interest Deduction Takers Switching to Standard', [])
    quantile_dict['Number of Interest Deduction Takers Switching to Standard'].append(bindf['switch'].sum())
    quantile_dict.setdefault('Number of Interest Deduction Takers', [])
    quantile_dict['Number of Interest Deduction Takers'].append(bindf['intr'].sum())
quantile_df = pd.DataFrame(quantile_dict)
quantile_df = quantile_df.append(quantile_df.sum(numeric_only=True), ignore_index=True)

quantile_df.index = list(zip(range(1,nbins + 1), 
                           sdf.groupby('bins')['c00100'].min(), 
                           sdf.groupby('bins')['c00100'].max())) + ['Total (Ignore Share Total)']
quantile_df

,Number of Interest Deduction Takers,Number of Interest Deduction Takers Switching to Standard,Share of Interest Deduction Takers Switching to Standard
"(1, 0.496695789163, 4377.20002579)",0.00,0.00,0.00
"(2, 4377.62818215, 10925.2370646)","13,339.02","3,221.42",0.24
"(3, 10925.2370646, 17561.6869681)","290,814.17","202,190.37",0.70
"(4, 17561.6869681, 26551.5962828)","611,190.42","411,484.44",0.67
"(5, 26551.5986875, 37445.9949212)","1,256,671.79","861,560.53",0.69
"(6, 37445.9949212, 51522.1260651)","2,293,844.22","1,566,412.85",0.68
"(7, 51523.6124798, 69819.6364485)","4,001,049.69","2,470,908.23",0.62
"(8, 69820.5672485, 98062.4259455)","6,115,719.36","3,642,949.18",0.60
"(9, 98065.2950764, 148003.050189)","9,133,761.45","5,453,520.68",0.60
"(10, 148003.050189, 197584873.031)","12,170,547.80","4,642,295.99",0.38


## Average Change in Tax Liability for Mortgage Deduction Takers

In [27]:
smdf = copy.deepcopy(sdf[sdf['switch'] > 0])
smdf['iichange'] = smdf['newiitax'] - smdf['iitax']
print('Average Change in Tax Liability for Mortgage Deduction Takers Switching to Standard: ' + str(w_avg(smdf, 'iichange')))

Average Change in Tax Liability for Mortgage Deduction Takers Switching to Standard: -1957.2256489914853
